# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [3]:
import pandas as pd

# URL of the raw CSV file
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"

# Read the CSV file from the URL
insurance_data = pd.read_csv(url)

url2 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
insurance_data2 = pd.read_csv(url2)

url3 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"
insurance_data3 = pd.read_csv(url3)

merged_data = pd.concat([insurance_data, insurance_data2, insurance_data3], ignore_index=True)

merged_data

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount,State,Gender
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934,NaN,NaN
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935,NaN,NaN
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247,NaN,NaN
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344,NaN,NaN
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12069,LA72316,NaN,NaN,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764,California,M
12070,PK87824,NaN,NaN,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000,California,F
12071,TD14365,NaN,NaN,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983,California,M
12072,UP19263,NaN,NaN,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000,California,M


In [4]:
# Function to clean column names
def clean_column_names(merged_data):
    column_names = merged_data.columns.tolist()
    column_names = [col.lower().replace(" ", "_") for col in column_names]
    column_names = [
        "state" if col == "st" else col for col in column_names
    ]
    merged_data.columns = column_names
    return merged_data

# Function to clean gender values
def clean_gender(merged_data):
    merged_data["gender"] = merged_data["gender"].str.lower().str.strip()
    merged_data["gender"] = merged_data["gender"].apply(lambda x: "F" if x.startswith("f") else "M")
    return merged_data

# Function to clean state values
def clean_state(merged_data):
    merged_data['state'] = merged_data['state'].str.upper()
    merged_data.loc[merged_data['state'] == 'AZ', 'state'] = 'ARIZONA'
    merged_data.loc[merged_data['state'] == 'CALI', 'state'] = 'CALIFORNIA'
    merged_data.loc[merged_data['state'] == 'WA', 'state'] = 'WASHINGTON'
    return merged_data

# Function to clean education values
def clean_education(merged_data):
    merged_data.loc[merged_data['education'] == 'Bachelors', 'education'] = 'Bachelor'
    return merged_data

# Function to clean customer lifetime value
def clean_customer_lifetime_value(merged_data):
    merged_data['customer_lifetime_value'] = merged_data['customer_lifetime_value'].str.replace('%', '')
    merged_data['customer_lifetime_value'] = pd.to_numeric(merged_data['customer_lifetime_value'], errors='coerce')
    return merged_data

# Function to clean vehicle class values
def clean_vehicle_class(merged_data):
    merged_data.loc[merged_data['vehicle_class'] == 'Sports Car', 'vehicle_class'] = 'Luxury'
    merged_data.loc[merged_data['vehicle_class'] == 'Luxury SUV', 'vehicle_class'] = 'Luxury'
    merged_data.loc[merged_data['vehicle_class'] == 'Luxury Car', 'vehicle_class'] = 'Luxury'
    return merged_data

# Function to clean number of open complaints
def clean_complaints(merged_data):
    merged_data['number_of_open_complaints'] = merged_data['number_of_open_complaints'].str.split('/').str[1]
    merged_data['number_of_open_complaints'] = pd.to_numeric(merged_data['number_of_open_complaints'], errors='coerce')
    return merged_data

# Function to handle null values
def handle_nulls(merged_data):
    merged_data.dropna(subset=['customer'], inplace=True)
    categorical_columns = ['state', 'gender', 'education', 'policy_type', 'vehicle_class']
    for col in categorical_columns:
        merged_data[col].fillna(merged_data[col].mode()[0], inplace=True)
    numerical_columns = ['customer_lifetime_value', 'income', 'monthly_premium_auto',
                         'number_of_open_complaints', 'total_claim_amount']
    for col in numerical_columns:
        merged_data[col].fillna(insurance_data[col].median(), inplace=True)
    return merged_data

# Function to handle duplicates
def remove_duplicates(merged_data):
    merged_data.drop_duplicates(inplace=True)
    merged_data.reset_index(drop=True, inplace=True)
    return merged_data

# Main function to apply all cleaning functions
def main(merged_data):
    # Load the dataset
    merged_data = pd.concat([insurance_data, insurance_data2, insurance_data3], ignore_index=True)
        # Return the cleaned dataframe
    return merged_data

In [14]:
merged_data

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount,State,Gender
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934,NaN,NaN
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935,NaN,NaN
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247,NaN,NaN
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344,NaN,NaN
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12069,LA72316,NaN,NaN,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764,California,M
12070,PK87824,NaN,NaN,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000,California,F
12071,TD14365,NaN,NaN,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983,California,M
12072,UP19263,NaN,NaN,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000,California,M


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [16]:
# URL of the raw CSV file
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"

# Read the CSV file from the URL
marketing_customer_data = pd.read_csv(url)

marketing_customer_data

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,...,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,1
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,2011-01-06,Employed,F,...,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,1
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,...,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,A,2
10908,10908,WA60547,California,11971.977650,No,Premium,College,2011-02-13,Employed,F,...,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2


1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [17]:
total_revenue_by_channel = marketing_customer_data.pivot_table(index='sales_channel', values='customer_lifetime_value', aggfunc='sum').round(2)
total_revenue_by_channel

,customer_lifetime_value
sales_channel,
Agent,33057887.85
Branch,24359201.21
Call Center,17364288.37
Web,12697632.90


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [18]:
average_lifetime_value_by_gender_and_education = marketing_customer_data.pivot_table(index=['gender', 'education'], values='customer_lifetime_value', aggfunc='mean').round(2)
average_lifetime_value_by_gender_and_education

customer_lifetime_value
gender education                                    
F      Bachelor                              7874.27
       College                               7748.82
       Doctor                                7328.51
       High School or Below                  8675.22
       Master                                8157.05
M      Bachelor                              7703.60
       College                               8052.46
       Doctor                                7415.33
       High School or Below                  8149.69
       Master                                8168.83

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [20]:
number_of_complaints_by_policy_type_and_month = marketing_customer_data.pivot_table(index=['policy_type', 'month'], values='number_of_open_complaints', aggfunc='sum')
number_of_complaints_by_policy_type_and_month

number_of_open_complaints
policy_type    month                           
Corporate Auto 1                     443.434952
               2                     385.208135
Personal Auto  1                    1727.605722
               2                    1453.684441
Special Auto   1                      87.074049
               2                      95.226817

In [21]:
long_format_table = number_of_complaints_by_policy_type_and_month.reset_index()
long_format_table

,policy_type,month,number_of_open_complaints
0,Corporate Auto,1,443.434952
1,Corporate Auto,2,385.208135
2,Personal Auto,1,1727.605722
3,Personal Auto,2,1453.684441
4,Special Auto,1,87.074049
5,Special Auto,2,95.226817
